# Install Required Libraries

In [ ]:
%pip install langchain==0.3.14
%pip install langchain-openai==0.2.14
%pip install pymongo==4.10.1

# Load Environment Variables 

In [ ]:
# Create .env file if it doesn't exist
%cp -n .env.example .env

In [1]:
# load the environment variables from .env file
import os

from dotenv import load_dotenv

load_dotenv(".env", override=True)

True

# Initialize OpenAI Client

Save the `api_type`, `base_url`, `api_version`, and `api_key` as global variables to avoid the need to supply them later in code.

In [2]:
import openai

openai.api_type = os.getenv("OPENAI_API_TYPE", "azure")
openai.base_url = os.getenv("AZURE_OPENAI_ENDPOINT", "https://<YOUR-OPENAI-DEPLOYMENT-NAME>.openai.azure.com/")
openai.api_version = os.getenv("OPENAI_API_VERSION", "2023-09-15-preview")
openai.api_key = os.getenv("OPENAI_API_KEY", "<YOUR-DEPLOYMENT-KEY>")

# Intialize the MongoDB Client

In [3]:
from urllib.parse import quote_plus

from pymongo import MongoClient

# Read and Store Environment variables
mongo_connection_string = os.getenv("AZURE_COSMOS_CONNECTION_STRING", "<YOUR-COSMOS-DB-CONNECTION-STRING>")
mongo_username = quote_plus(os.getenv("AZURE_COSMOS_USERNAME"))
mongo_password = quote_plus(os.getenv("AZURE_COSMOS_PASSWORD"))
mongo_connection_string = mongo_connection_string.replace("<user>", mongo_username).replace(
    "<password>", mongo_password
)

collection_name = os.getenv("AZURE_COSMOS_COLLECTION_NAME", "collectionName")
database_name = os.getenv("AZURE_COSMOS_DATABASE_NAME", "DatabaseName")

# Initialize the MongoClient
mongo_client = MongoClient(mongo_connection_string)

# Create the database if it doesn't exist
db = mongo_client[database_name]

# Create the collection if it doesn't exist
collection = db[collection_name]

/var/folders/db/x_x115ns61154jxycm1mwr780000gn/T/ipykernel_54773/1277193319.py:17: UserWarning: You appear to be connected to a CosmosDB cluster. For more information regarding feature compatibility and support please visit https://www.mongodb.com/supportability/cosmosdb
  mongo_client = MongoClient(mongo_connection_string)


# Load JSON Data

In [4]:
import json

from langchain.docstore.document import Document

SOURCE_FILE_NAME = "./src/data/food_items.json"


def read_data(file_path) -> list[Document]:
    # Load JSON file
    with open(file_path) as file:
        json_data = json.load(file)

    documents = []
    absolute_path = os.path.abspath(file_path)
    # Process each item in the JSON data
    for idx, item in enumerate(json_data):
        documents.append(
            Document(page_content=json.dumps(item), metadata={"source": absolute_path, "seq_num": idx + 1})
        )

    return documents

In [5]:
json_data = read_data(SOURCE_FILE_NAME)

In [6]:
# Display a sample from the data
print(json_data[1])

page_content='{"category": "Smoothies", "name": "Jimmy Jam Smoothie", "description": "Berries n kale, strawberries, bananas, blueberries kale, tropical fruit blend, and dragon fruit. Our fruity tasty smoothies are blended to perfection.", "price": "5.49 USD"}' metadata={'source': '/Users/john0isaac/Developer/Cosmic-Food-RAG-app/src/data/food_items.json', 'seq_num': 2}


# Initialize the Embeddings Client

In [7]:
from langchain_openai import AzureOpenAIEmbeddings

openai_embeddings_model = os.getenv("AZURE_OPENAI_EMBEDDINGS_MODEL_NAME", "text-embedding-ada-002")
openai_embeddings_deployment = os.getenv("AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT_NAME", "text-embedding")

azure_openai_embeddings: AzureOpenAIEmbeddings = AzureOpenAIEmbeddings(
    model=openai_embeddings_model,
    azure_deployment=openai_embeddings_deployment,
)

# Generate and Save Embeddings to MongoDB

In [8]:
from langchain_community.vectorstores.azure_cosmos_db import AzureCosmosDBVectorSearch

index_name = os.getenv("AZURE_COSMOS_INDEX_NAME", "indexName")

# Only Run this the first time you open the notebook
# Create embeddings from the data, save to the database and return a connection to MongoDB vCore
vector_store: AzureCosmosDBVectorSearch = AzureCosmosDBVectorSearch.from_documents(
    json_data,
    azure_openai_embeddings,
    collection=collection,
    index_name=index_name,
)

In [ ]:
from langchain_community.vectorstores.azure_cosmos_db import AzureCosmosDBVectorSearch

# Run this to connect to the vector store
vector_store: AzureCosmosDBVectorSearch = AzureCosmosDBVectorSearch.from_connection_string(
    connection_string=mongo_connection_string,
    namespace=f"{database_name}.{collection_name}",
    embedding=azure_openai_embeddings,
)

# Create Vector Index (HNSW)

In [9]:
from langchain_community.vectorstores.azure_cosmos_db import (
    CosmosDBSimilarityType,
    CosmosDBVectorSearchType,
)

# Read more about these variables in detail here. https://learn.microsoft.com/en-us/azure/cosmos-db/mongodb/vcore/vector-search
num_lists = 100
dimensions = 1536
similarity_algorithm = CosmosDBSimilarityType.COS
kind = CosmosDBVectorSearchType.VECTOR_HNSW
m = 16
ef_construction = 64

# Create the collection and the index
vector_store.create_index(num_lists, dimensions, similarity_algorithm, kind, m, ef_construction)

{'raw': {'defaultShard': {'numIndexesBefore': 1,
   'numIndexesAfter': 2,
   'createdCollectionAutomatically': False,
   'ok': 1}},
 'ok': 1}

## Test Vector Search Flow

In [10]:
query = "Beef Bacon"
docs = vector_store.similarity_search(query)
print(docs[0].page_content)

{"category": "Sandwiches", "name": "Bacon Turkey Bravo Sandwich", "description": "Whole (1010 Cal.), Half (500 Cal.) Oven-roasted turkey breast raised without antibiotics, Applewood-smoked bacon, smoked Gouda, emerald greens, vine-ripened tomatoes, signature sauce , salt and pepper on Tomato Basil Bread. Allergens: Contains Wheat, Milk, Egg", "price": "8.79 USD"}


# Initialize the Chat Client

In [11]:
from langchain_openai import AzureChatOpenAI

openai_chat_model = os.getenv("AZURE_OPENAI_CHAT_MODEL_NAME", "gpt-35-turbo")
openai_chat_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "chat-gpt")

azure_openai_chat: AzureChatOpenAI = AzureChatOpenAI(
    model=openai_chat_model,
    azure_deployment=openai_chat_deployment,
)

In [12]:
# Test the chat flow
chat_response = azure_openai_chat.invoke("Tell me a joke")
print(chat_response.content)

Why don't scientists trust atoms?

Because they make up everything!


In [13]:
chat_response = azure_openai_chat.astream("Write a 200 words essay about education.")

async for response in chat_response:
    print(response.content, end="")

Education is a fundamental aspect of human development. It plays a pivotal role in shaping individuals' minds, fostering critical thinking skills, and equipping them with the necessary knowledge and skills to succeed in life. Education is not confined to the walls of a classroom; it encompasses a broader spectrum of learning experiences that occur throughout a person's lifetime.

Firstly, education provides individuals with a solid foundation of knowledge. From basic literacy and numeracy skills to more advanced subjects like science, history, and literature, education broadens our understanding of the world around us. It helps us make informed decisions and enables us to contribute meaningfully to society.

Furthermore, education fosters critical thinking and problem-solving skills. It encourages individuals to question, analyze, and think independently. Through various teaching methods such as discussions, research projects, and practical applications, education promotes creativity a

# Create RAG Function

In [14]:
from langchain.prompts import ChatPromptTemplate

REPHRASE_PROMPT = """\
Given the following conversation and a follow up question, rephrase the follow up \
question to be a standalone question.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone Question:"""

CONTEXT_PROMPT = """\
You are a restaurant chatbot, tasked with answering any question about \
food dishes from the contex.

Generate a response of 80 words or less for the \
given question based solely on the provided search results (name, description, price, and category). \
You must only use information from the provided search results. Use an unbiased and \
fun tone. Do not repeat text. Your response must be solely based on the provided context.

If there is nothing in the context relevant to the question at hand, just say "Hmm, \
I'm not sure." Don't try to make up an answer.

Anything between the following `context` html blocks is retrieved from a knowledge \
bank, not part of the conversation with the user. 

<context>
    {context} 
<context/>

REMEMBER: If there is no relevant information within the context, just say "Hmm, I'm \
not sure." Don't try to make up an answer. Anything between the preceding 'context' \
html blocks is retrieved from a knowledge bank, not part of the conversation with the \
user.\

User Question: {input}

Chatbot Response:"""

rephrase_prompt_template = ChatPromptTemplate.from_template(REPHRASE_PROMPT)
context_prompt_template = ChatPromptTemplate.from_template(CONTEXT_PROMPT)

In [15]:
limit = 3
score_threshold = 0.5
search_type = "similarity"

In [16]:
from quartapp.approaches.rag import get_data_points

# Vector Store Retriever
vector_store_retriever = vector_store.as_retriever(
    search_type=search_type, search_kwargs={"k": limit, "score_threshold": score_threshold}
)
# Rephrase Chain
rephrase_chain = rephrase_prompt_template | azure_openai_chat
# Context Chain
context_chain = context_prompt_template | azure_openai_chat

## Test RAG Flow

In [17]:
# 1. Rephrase the question
messages = [{"content": "Do you have any vegan options?", "role": "user"}]

rephrased_question = rephrase_chain.invoke({"chat_history": messages[:-1], "question": messages[-1]})
print(rephrased_question.content)

What vegan options do you have?


In [18]:
# 2. Get the context from the database and format it to remove the embeddings
vector_context = vector_store_retriever.invoke(str(rephrased_question.content))
data_points = get_data_points(vector_context)
print(data_points)

[DataPoint(name='Beyond Burger', description='Served with Romaine lettuce, tomato, pickle, vegan mayonnaise, ketchup, and mustard on a toasted bun. Sandwich made with whole wheat bread. Can be made as a wrap in a whole wheat tortilla. Served with kettle potato chips or corn tortilla chips.', price='9.0 USD', category='Sandwiches', collection=None), DataPoint(name='Tofu Salad Sandwich', description='Served with Romaine lettuce, tomato, vegan mayonnaise, and mustard. Sandwich made with whole wheat bread. Can be made as a wrap in a whole wheat tortilla. Served with kettle potato chips or corn tortilla chips.', price='9.0 USD', category='Sandwiches', collection=None)]


In [19]:
# 3. Generate a response based on the context
response = context_chain.invoke({"context": [dp.to_dict() for dp in data_points], "input": rephrased_question.content})
print(response.content)

We have a couple of delicious vegan options for you! Our Beyond Burger is made with plant-based ingredients and served with lettuce, tomato, pickle, vegan mayo, ketchup, and mustard on a toasted bun. We also have a Tofu Salad Sandwich with lettuce, tomato, vegan mayo, and mustard. Both sandwiches can be made as wraps and come with kettle potato chips or corn tortilla chips. They are priced at $9. Enjoy your vegan meal!


In [20]:
# 4. Store the chat history and the response
messages.append({"content": response.content, "role": "assistant"})

In [21]:
# Test with another question to see if the chat history is maintained
messages.append({"content": "what is the price of the first dish?", "role": "user"})

rephrased_question = rephrase_chain.invoke({"chat_history": messages[:-1], "question": messages[-1]})
vector_context = vector_store_retriever.invoke(str(rephrased_question.content))
data_points = get_data_points(vector_context)
response = context_chain.invoke({"context": [dp.to_dict() for dp in data_points], "input": rephrased_question.content})

In [22]:
print("Rephrased Question: ", rephrased_question.content)
print("LLM Response: ", response.content)

Rephrased Question:  What is the price of the first vegan dish?
LLM Response:  The price of the first vegan dish, Veggie Samosa, is 8.35 USD. Enjoy these triangular pastries filled with spiced potatoes, peas, and lentils, served with a side of mint or tamarind relish!


# Inline Embeddings Generation

In [ ]:
mongo_connection_string = os.getenv("AZURE_COSMOS_CONNECTION_STRING_AUTO_EMBEDDING", "conn_string")
mongo_client = MongoClient(mongo_connection_string)

db_name = os.getenv("AZURE_COSMOS_DATABASE_NAME", "DatabaseName")
db = mongo_client[db_name]

collection_name = os.getenv("AZURE_COSMOS_COLLECTION_NAME", "collectionName")
index_name = os.getenv("AZURE_COSMOS_INDEX_NAME", "indexName")

collection = db[collection_name]

# Insert data
docs = [json.loads(item.page_content) for item in json_data[0:20]]
for doc in docs:
    collection.insert_one(doc)

# Inline generate embeddings
collection.update_many({}, {"$generateEmbeddings": {"description": "embeddings"}})

# Create IVF index
createIndexCommand = {
    "createIndexes": collection_name,
    "indexes": [
        {
            "key": {"embeddings": "cosmosSearch"},
            "name": "ivf_index",
            "cosmosSearchOptions": {
                "kind": "vector-ivf",
                "m": 4,
                "efConstruction": 16,
                "similarity": "COS",
                "dimensions": 1536,
            },
        }
    ],
}
db.command(createIndexCommand)

In [ ]:
search_pipeline = [
    {"$search": {"cosmosSearch": {"query": docs[0]["description"], "k": 5, "path": "embeddings", "efSearch": 100}}},
    {"$project": {"similarityScore": {"$meta": "searchScore"}, "_id": 0, "name": 1, "description": 1}},
]

results = collection.aggregate(search_pipeline)

for result in results:
    print(f"[Score: {result['similarityScore']:.3f}] {result['name']}: {result['description']}")